# Sales Report using SQL + Pandas 📊

## Install & import the dependencies

In [1]:
#  import sys
# !{sys.executable} -m pip install fpdf -q
# !{sys.executable} -m pip install pandas -q
# !{sys.executable} -m pip install plotly-express -q
# !{sys.executable} -m 
# pip install kaleido==0.1.0post1

In [2]:
# pip install -r requirements.txt

In [3]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express kaleido
from fpdf import FPDF  # pip install fpdf

## Define paths and chart style

In [4]:
# Define the plotly template. Some other examples:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

In [5]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

## Total Sales by Month

In [6]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [7]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [8]:
# Print the DataFrame
print(df)

      sale_date  total_sales
0    2022-01-01          975
1    2022-01-02          150
2    2022-01-03          720
3    2022-01-04          250
4    2022-01-05         1375
..          ...          ...
334  2022-12-27          520
335  2022-12-28          250
336  2022-12-29          680
337  2022-12-30          610
338  2022-12-31          655

[339 rows x 2 columns]


In [9]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    339 non-null    object
 1   total_sales  339 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [10]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    339 non-null    datetime64[ns]
 1   total_sales  339 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.4 KB


In [11]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

total_sales
sale_date              
2022-01-01          975
2022-01-02          150
2022-01-03          720

In [12]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

total_sales
sale_date              
2022-01-31        25580
2022-02-28        20465
2022-03-31        25625
2022-04-30        22745
2022-05-31        26025
2022-06-30        24985
2022-07-31        20520
2022-08-31        20850
2022-09-30        21565
2022-10-31        25120
2022-11-30        26375
2022-12-31        16820

In [13]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

total_sales month_name
sale_date                         
2022-01-31        25580        Jan
2022-02-28        20465        Feb
2022-03-31        25625        Mar
2022-04-30        22745        Apr
2022-05-31        26025        May
2022-06-30        24985        Jun
2022-07-31        20520        Jul
2022-08-31        20850        Aug
2022-09-30        21565        Sep
2022-10-31        25120        Oct
2022-11-30        26375        Nov
2022-12-31        16820        Dec

In [14]:
import plotly
print(plotly.__version__)

5.9.0


In [15]:
# pip install --force-reinstall kaleido==0.1.0post1

In [16]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
# fig.write_image(output_dir / 'monthly_sales.png')

## Total Sales by Product

In [17]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)

In [18]:
# Print the DataFrame
print(df)

  product_name  total_sales
0    Product A        57700
1    Product B        26375
2    Product C        84900
3    Product D        45720
4    Product E        61980


In [ ]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Productt',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.jpeg')

## Top Customer by Sales

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [ ]:
df

In [ ]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

## Create PDF Report

In [ ]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

##  SQL vs. Pandas

### Customer Segment using SQL

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

### Customer Segment using SQL + Pandas

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

In [ ]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

In [ ]:
# Close the connection
conn.close()